In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
file_list = [f for f in os.listdir('/Users/zihangwei/Library/CloudStorage/OneDrive-TexasA&MUniversity/Publishing_Paper/PDE_Network_Modeling/2_RawData') if f.endswith('.txt')]

In [3]:
delimiter = ','  # Change this to the actual delimiter if it's different
data_all = pd.DataFrame()
for file_path in file_list:
    # Read the text file into a Pandas DataFrame
    df = pd.read_csv('/Users/zihangwei/Library/CloudStorage/OneDrive-TexasA&MUniversity/Publishing_Paper/PDE_Network_Modeling/2_RawData/' + file_path, delimiter=delimiter, header=None)
    df = df.iloc[:, :12]
    names = ['TimeStamp', 'Station', 'District', 'Freeway_#', 'Dir_Travel', 'Type', 'Length', 'Samples',
             'Observed_%', 'Total_Flow', 'Occupancy', 'Speed']
    df.columns = names
    df = df[df['Type']=="ML"]

    data_all = pd.concat([data_all, df], ignore_index=True)

In [4]:
data_all_observed = data_all[data_all['Observed_%'] > 0]
summary_observed = data_all_observed.groupby('Station').size().reset_index(name='Count')

In [5]:
metadata = pd.read_csv('Data_Raw/Archive/d07_text_meta_2022_07_15.txt', delimiter='\t')
metadata = metadata[metadata['Type']=="ML"]

In [6]:
metadata_obs = pd.merge(metadata, summary_observed, left_on='ID', right_on='Station' , how='left')

In [7]:
#metadata_obs.to_csv('Data_Raw/Station_MetaData.csv', index=False)

## Station Selection

In [32]:
station_selected = pd.read_csv('Data_Raw/I605_Exported.csv')
station_selected = station_selected[station_selected['Count'] == 2016]

In [33]:
station_selected['PM_Dir'] = station_selected['Abs_PM'] - station_selected['Abs_PM'].min()
#station_selected['PM_Dir_round0.5'] = station_selected['PM_Dir'].apply(lambda x: round(x * 2) / 2)
station_selected['Station_PM'] = station_selected['PM_Dir'].round(0)
station_selected['PM_Deviation'] = abs(station_selected['PM_Dir'] - station_selected['Station_PM'])

In [34]:
# select the stations where the deviation from integer mile is the smallest.
min_index = station_selected.groupby('Station_PM')['PM_Deviation'].idxmin()
station_selected = station_selected.loc[min_index]

In [35]:
station_selected['Station_Number'] = station_selected['Station_PM'].rank(ascending=True)

In [36]:
station_selected

,ID,Fwy,Dir,District,County,City,State_PM,Abs_PM,Latitude,Longitude,...,User_ID_1,User_ID_2,User_ID_3,User_ID_4,Station,Count,PM_Dir,Station_PM,PM_Deviation,Station_Number
0,717831,605,N,7,37,70686.0,R.23,2.282,33.809588,-118.081304,...,3435,NaN,NaN,NaN,717831.0,2016.0,0.000,0.0,0.000,1.0
2,761844,605,N,7,37,43000.0,R1.03,3.082,33.821171,-118.081273,...,2137,NaN,NaN,NaN,761844.0,2016.0,0.800,1.0,0.200,2.0
6,716760,605,N,7,37,39892.0,R2.28,4.332,33.838278,-118.087885,...,2138,NaN,NaN,NaN,716760.0,2016.0,2.050,2.0,0.050,3.0
9,773736,605,N,7,37,12552.0,R3.3,5.352,33.851922,-118.094520,...,2660,NaN,NaN,NaN,773736.0,2016.0,3.070,3.0,0.070,4.0
12,717854,605,N,7,37,12552.0,R4.31,6.362,33.866120,-118.098646,...,2139,NaN,NaN,NaN,717854.0,2016.0,4.080,4.0,0.080,5.0
15,773749,605,N,7,37,12552.0,R5.2,7.252,33.878493,-118.103113,...,2439,NaN,NaN,NaN,773749.0,2016.0,4.970,5.0,0.030,6.0
19,716784,605,N,7,37,12552.0,R6.29,8.342,33.894297,-118.104669,...,2140,NaN,NaN,NaN,716784.0,2016.0,6.060,6.0,0.060,7.0
22,717864,605,N,7,37,52526.0,R7.33,9.382,33.909354,-118.104924,...,2141,NaN,NaN,NaN,717864.0,2016.0,7.100,7.0,0.100,8.0
24,718466,605,N,7,37,52526.0,R8.2,10.252,33.921930,-118.104318,...,2910,NaN,NaN,NaN,718466.0,2016.0,7.970,8.0,0.030,9.0
27,717879,605,N,7,37,19766.0,R9.75,11.802,33.942882,-118.095620,...,2454,NaN,NaN,NaN,717879.0,2016.0,9.520,10.0,0.480,10.0


In [37]:
order_match = station_selected[['ID', 'Station_PM']]

In [38]:
ID_selected = station_selected['ID']
data_all_select = data_all[data_all['Station'].isin(ID_selected)]
data_all_select['TimeStamp'] = pd.to_datetime(data_all_select['TimeStamp'])
data_all_select['TimeDiff'] = ((data_all_select['TimeStamp'] - data_all_select['TimeStamp'].min()).dt.total_seconds())/60
data_all_select['TimeStep'] = 1 + (data_all_select['TimeDiff'] // 5)

/var/folders/g5/yg5rt0h52kxc5z_0zd7jrqbh0000gn/T/ipykernel_40607/440614197.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_all_select['TimeStamp'] = pd.to_datetime(data_all_select['TimeStamp'])
/var/folders/g5/yg5rt0h52kxc5z_0zd7jrqbh0000gn/T/ipykernel_40607/440614197.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_all_select['TimeDiff'] = ((data_all_select['TimeStamp'] - data_all_select['TimeStamp'].min()).dt.total_seconds())/60
/var/folders/g5/yg5rt0h52kxc5z_0zd7jrqbh0000gn/T/ipykernel_4

In [39]:
#station_selected[station_selected['PM_Dir_round'].duplicated(keep=False)]

In [40]:
data_all_select = pd.merge(data_all_select, order_match, left_on='Station', right_on='ID' , how='left')

In [41]:
Flow = data_all_select[['TimeStep', 'Station_PM', 'Total_Flow']]
Flow['TimeStep'] = Flow['TimeStep'].astype(int)
Flow['Station_PM'] = Flow['Station_PM'].astype(int)

Occupancy = data_all_select[['TimeStep', 'Station_PM', 'Occupancy']]
Occupancy['TimeStep'] = Occupancy['TimeStep'].astype(int)
Occupancy['Station_PM'] = Occupancy['Station_PM'].astype(int)

Speed = data_all_select[['TimeStep', 'Station_PM', 'Speed']]
Speed['TimeStep'] = Speed['TimeStep'].astype(int)
Speed['Station_PM'] = Speed['Station_PM'].astype(int)

/var/folders/g5/yg5rt0h52kxc5z_0zd7jrqbh0000gn/T/ipykernel_40607/3535391722.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Flow['TimeStep'] = Flow['TimeStep'].astype(int)
/var/folders/g5/yg5rt0h52kxc5z_0zd7jrqbh0000gn/T/ipykernel_40607/3535391722.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Flow['Station_PM'] = Flow['Station_PM'].astype(int)
/var/folders/g5/yg5rt0h52kxc5z_0zd7jrqbh0000gn/T/ipykernel_40607/3535391722.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

## Final Data Cleaning

In [42]:
"""
Create a table with timestep and PM (posted Miles) as one-one match
"""
timestep_value = [i for i in range(1, Occupancy['TimeStep'].max() + 1) for _ in range(Occupancy['Station_PM'].max() + 1)]
PM_values = list(range(Occupancy['Station_PM'].max() + 1)) * Occupancy['TimeStep'].max()
Time_PM = pd.DataFrame({'TimeStep': timestep_value, 'Station_PM': PM_values})

In [43]:
Occupancy_all = pd.merge(Time_PM, Occupancy, on=['TimeStep', 'Station_PM'] , how='left')
Flow_all = pd.merge(Time_PM, Flow, on=['TimeStep', 'Station_PM'] , how='left')
Speed_all = pd.merge(Time_PM, Speed, on=['TimeStep', 'Station_PM'] , how='left')

In [44]:
Occupancy_all

,TimeStep,Station_PM,Occupancy
0,1,0,0.0132
1,1,1,0.0175
2,1,2,0.0198
3,1,3,0.0223
4,1,4,0.0214
...,...,...,...
50395,2016,20,0.0259
50396,2016,21,0.0256
50397,2016,22,0.0241
50398,2016,23,0.0232


In [45]:
Occupancy_all_wide = Occupancy_all.pivot(index='Station_PM', columns='TimeStep', values='Occupancy')
Flow_all_wide = Flow_all.pivot(index='Station_PM', columns='TimeStep', values='Total_Flow')
Speed_all_wide = Speed_all.pivot(index='Station_PM', columns='TimeStep', values='Speed')

In [46]:
Flow_wide = Flow.pivot(index='Station_PM', columns='TimeStep', values='Total_Flow')
Occupancy_wide = Occupancy.pivot(index='Station_PM', columns='TimeStep', values='Occupancy')
Speed_wide = Speed.pivot(index='Station_PM', columns='TimeStep', values='Speed')

In [47]:
Flow_all_wide.to_csv('Data_Clean/Flow_all.csv')
Occupancy_all_wide.to_csv('Data_Clean/Occupancy_all.csv')
Speed_all_wide.to_csv('Data_Clean/Speed_all.csv')

In [48]:
Flow_wide.to_csv('Data_Clean/Flow.csv')
Occupancy_wide.to_csv('Data_Clean/Occupancy.csv')
Speed_wide.to_csv('Data_Clean/Speed.csv')

In [49]:
Flow.to_csv('Data_Clean/Flow_Long.csv', index=False)
Occupancy.to_csv('Data_Clean/Occupancy_Long.csv', index=False)
Speed.to_csv('Data_Clean/Speed_Long.csv', index=False)

In [50]:
Flow_all.to_csv('Data_Clean/Flow_all_Long.csv', index=False)
Occupancy_all.to_csv('Data_Clean/Occupancy_all_Long.csv', index=False)
Speed_all.to_csv('Data_Clean/Speed_all_Long.csv', index=False)